# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, Flatten, Dense, Dropout, Reshape, MaxPool2D
from tensorflow.keras.callbacks import ModelCheckpoint

TensorFlow 2.x selected.


In [2]:
(trainX, trainY), (testX, testY) = cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [0]:
# Splitting data set into 2 categories one with 0-4 labels & other with 5-9 labels.

x_train_zero_four = []
y_train_zero_four = []
x_train_five_nine = []
y_train_five_nine = []

x_test_zero_four = []
y_test_zero_four = []
x_test_five_nine = []
y_test_five_nine = []

for idx,x in enumerate(trainY):
    if (x>4):
        x_train_five_nine.append(trainX[idx])
        y_train_five_nine.append(trainY[idx])
    else:
        x_train_zero_four.append(trainX[idx])
        y_train_zero_four.append(trainY[idx])

        
for idx,y in enumerate(testY):
    if (y>4):
        x_test_five_nine.append(testX[idx])
        y_test_five_nine.append(testY[idx])
    else:
        x_test_zero_four.append(testX[idx])
        y_test_zero_four.append(testY[idx])

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
y_train_zero_four = to_categorical(y_train_zero_four, num_classes=5)
y_test_zero_four = to_categorical(y_test_zero_four, num_classes=5)

In [5]:
y_train_zero_four[0]

array([0., 0., 0., 0., 1.], dtype=float32)

In [6]:
# printing shape of each data set post splitting

x_train_zero_four = np.asarray(x_train_zero_four,dtype='float')
y_train_zero_four = np.asarray(y_train_zero_four,dtype='float')
x_train_five_nine = np.asarray(x_train_five_nine,dtype='float')
y_train_five_nine = np.asarray(y_train_five_nine,dtype='float')
x_test_zero_four = np.asarray(x_test_zero_four,dtype='float')
y_test_zero_four = np.asarray(y_test_zero_four,dtype='float')
x_test_five_nine = np.asarray(x_test_five_nine,dtype='float')
y_test_five_nine = np.asarray(y_test_five_nine,dtype='float')
print(x_train_zero_four.shape)
print(y_train_zero_four.shape)
print(x_train_five_nine.shape)
print(y_train_five_nine.shape)
print(x_test_zero_four.shape)
print(y_test_zero_four.shape)
print(x_test_five_nine.shape)
print(y_test_five_nine.shape)

(25000, 32, 32, 3)
(25000, 5)
(25000, 32, 32, 3)
(25000, 1)
(5000, 32, 32, 3)
(5000, 5)
(5000, 32, 32, 3)
(5000, 1)


In [0]:
#Reshaping data

x_train_zero_four = x_train_zero_four.reshape(x_train_zero_four.shape[0], 32, 32, 3).astype('float32')
x_test_zero_four = x_test_zero_four.reshape(x_test_zero_four.shape[0], 32, 32, 3).astype('float32')

In [0]:
#Normalise data

x_train_zero_four /= 255
x_test_zero_four /= 255

In [9]:
print(x_train_zero_four.shape)
print(y_test_zero_four.shape)

(25000, 32, 32, 3)
(5000, 5)


### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [0]:
#Clear out tensorflow memory
tf.keras.backend.clear_session()

In [0]:
def create_CNN_model():
    #Initialize Sequential model
    model = Sequential()

    #normalize data
    model.add(BatchNormalization())

    #Add first convolutional layer
    model.add(Conv2D(32, #Number of filters 
                                    kernel_size=(3,3), #Size of the filter
                                    activation='relu'))

    #Add second convolutional layer
    model.add(Conv2D(32, #Number of filters 
                                    kernel_size=(3,3), #Size of the filter
                                    activation='relu'))

    #Add MaxPooling layer
    model.add(MaxPool2D(pool_size=(2,2)))

    #Flatten the output
    model.add(Flatten())

    #Dense layer
    model.add(Dense(200, activation='relu'))

    #Add another dropout layer
    model.add(Dropout(0.25))

    #Dense layer
    model.add(Dense(100, activation='relu'))

    #Add another dropout layer
    model.add(Dropout(0.25))

    #Output layer
    model.add(Dense(5, activation='softmax'))

    return model

In [0]:
model = create_CNN_model()

In [0]:
#Saving the best model using model checkpoint callback
model_checkpoint = ModelCheckpoint('flowertransfer_learning_ext_cnn.h5', 
                                    save_best_only=True, 
                                    monitor='val_acc', 
                                    mode='max', 
                                    verbose=1)

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [15]:

model.fit(x_train_zero_four, y_train_zero_four, batch_size=32, epochs=30, validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/30
20000/20000 [==============================] - 11s 528us/sample - loss: 1.0285 - accuracy: 0.5849 - val_loss: 0.8148 - val_accuracy: 0.6782
Epoch 2/30
20000/20000 [==============================] - 3s 149us/sample - loss: 0.7630 - accuracy: 0.7088 - val_loss: 0.7167 - val_accuracy: 0.7248
Epoch 3/30
20000/20000 [==============================] - 3s 153us/sample - loss: 0.6031 - accuracy: 0.7711 - val_loss: 0.6742 - val_accuracy: 0.7506
Epoch 4/30
20000/20000 [==============================] - 3s 143us/sample - loss: 0.4895 - accuracy: 0.8188 - val_loss: 0.6497 - val_accuracy: 0.7642
Epoch 5/30
20000/20000 [==============================] - 3s 146us/sample - loss: 0.3684 - accuracy: 0.8658 - val_loss: 0.7236 - val_accuracy: 0.7592
Epoch 6/30
20000/20000 [==============================] - 3s 145us/sample - loss: 0.2963 - accuracy: 0.8906 - val_loss: 0.7351 - val_accuracy: 0.7678
Epoch 7/30
20000/20000 [==========================

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

In [0]:
#Freezing layers in the model which don't have 'dense' in their name
for layer in model.layers:
  if('dense' not in layer.name): #prefix detection to freeze layers which does not have dense
    #Freezing a layer
    layer.trainable = False

### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

In [0]:
#Reshape
x_train_five_nine = x_train_five_nine.reshape(x_train_five_nine.shape[0], 32, 32, 3).astype('float32')
x_test_five_nine = x_test_five_nine.reshape(x_test_five_nine.shape[0], 32, 32, 3).astype('float32')

In [0]:
x_train_five_nine /= 255
x_test_five_nine /= 255

In [19]:
# encoding the labels for 5-9 of cifar 10 data set
y_train_five_nine = y_train_five_nine -5
y_test_five_nine = y_test_five_nine - 5

print(y_test_five_nine[0])

y_train_five_nine = to_categorical(y_train_five_nine, 5)
y_test_five_nine = to_categorical(y_test_five_nine, 5)

[3.]


In [20]:
#Transfer learning . Fit the model built on 0-4 data onto data 5-9
model.fit(x_train_five_nine, y_train_five_nine, batch_size=32, epochs=20,validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 3s 168us/sample - loss: 1.0245 - accuracy: 0.6309 - val_loss: 0.5913 - val_accuracy: 0.7842
Epoch 2/20
20000/20000 [==============================] - 3s 148us/sample - loss: 0.5381 - accuracy: 0.8045 - val_loss: 0.4826 - val_accuracy: 0.8300
Epoch 3/20
20000/20000 [==============================] - 3s 148us/sample - loss: 0.4091 - accuracy: 0.8544 - val_loss: 0.4833 - val_accuracy: 0.8222
Epoch 4/20
20000/20000 [==============================] - 3s 142us/sample - loss: 0.3207 - accuracy: 0.8870 - val_loss: 0.5507 - val_accuracy: 0.8256
Epoch 5/20
20000/20000 [==============================] - 3s 140us/sample - loss: 0.2587 - accuracy: 0.9084 - val_loss: 0.4972 - val_accuracy: 0.8410
Epoch 6/20
20000/20000 [==============================] - 3s 142us/sample - loss: 0.2017 - accuracy: 0.9270 - val_loss: 0.5366 - val_accuracy: 0.8394
Epoch 7/20
20000/20000 [===========================

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [3]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [5]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [6]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [7]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [0]:
# TfidfVectorizer
vect = TfidfVectorizer()

In [33]:
pd.DataFrame(vect.fit_transform(twenty_train).toarray(), columns=vect.get_feature_names())

,data,descr,filenames,target,target_names
0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0


In [0]:
vectorizer = TfidfVectorizer(min_df = 4,ngram_range = (1,2),stop_words = "english")
X_train = vectorizer.fit_transform(twenty_train.data)
X_test = vectorizer.transform(twenty_test.data)

In [12]:
pd.DataFrame(X_train.toarray(), columns=vectorizer.get_feature_names()).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2217,2218,2219,2220,2221,2222,2223,2224,2225,2226,2227,2228,2229,2230,2231,2232,2233,2234,2235,2236,2237,2238,2239,2240,2241,2242,2243,2244,2245,2246,2247,2248,2249,2250,2251,2252,2253,2254,2255,2256
00,0.0,0.0,0.0,0.0,0.057652,0.043053,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00 00,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00 08,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00 09,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00 34,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zoerasterism,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zoerasterism shintoism,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zoom,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zyeh,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [42]:
from sklearn.linear_model import LogisticRegression

LRmodel = LogisticRegression()
LRmodel.fit(X_train,twenty_train.target)
LR_score_train = LRmodel.score(X_train,twenty_train.target)
print("training score",LR_score_train)
LR_score_test = LRmodel.score(X_test,twenty_test.target)
print("test score",LR_score_test)

training score 0.9955693398316349
test score 0.9067909454061251
